<a href="https://colab.research.google.com/github/salihdegrli/sms_spam/blob/main/mBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers datasets torch scikit-learn

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from string import punctuation
import os
import re

In [ ]:
#@title UTILS
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

def predict_message(message):
    inputs = tokenizer(message,
                       truncation=True,
                       padding=True,
                       max_length=128,
                       return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_label = torch.argmax(probabilities, dim=1).item()
    return "Spam" if predicted_label == 1 else "Ham"

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # 😀-🙏 (Smileys & Emotion)
        u"\U0001F300-\U0001F5FF"  # 🌀-🛿 (Symbols & Pictographs)
        u"\U0001F680-\U0001F6FF"  # 🚀-🛺 (Transport & Map)
        u"\U0001F700-\U0001F77F"  # 🜀-🝿 (Alchemical Symbols)
        u"\U0001F780-\U0001F7FF"  # 🞀-🟿 (Geometric Shapes Extended)
        u"\U0001F800-\U0001F8FF"  # 🠀-🢿 (Supplemental Arrows)
        u"\U0001F900-\U0001F9FF"  # 🤀-🧿 (Supplemental Symbols and Pictographs)
        u"\U0001FA00-\U0001FA6F"  # 🨀-🩯 (Chess Symbols & Objects)
        u"\U0001FA70-\U0001FAFF"  # 🩰-🫿 (Extended Pictographs)
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

##read and preprocess data

In [ ]:
df = pd.read_csv("/content/sms_spam_train.csv")
df = df.sample(frac=1).reset_index(drop=True)
df.columns = [col.lower() for col in df.columns]

df['label'] = df['label'].map({'ham': 0, 'spam': 1})
df['message'] = df['message'].str.lower()
df['message'] = df['message'].apply(lambda x: re.sub(r'http\S+|www\S+|\.com', '', x))
df['message'] = df['message'].apply(lambda x: re.sub(r'\s+', ' ', x))
df['message'] = df['message'].apply(lambda x: x.strip())
df['message'] = df['message'].apply(lambda x: remove_emojis(x))

##dataset loader

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

train_messages, test_messages, train_labels, test_labels = train_test_split(df['message'].tolist(),
                                                                            df['label'].tolist(),
                                                                            test_size=0.2,
                                                                            random_state=60)
train_encodings = tokenizer(train_messages,
                            truncation=True,
                            padding=True,
                            max_length=128
                            )

test_encodings = tokenizer(test_messages,
                           truncation=True,
                           padding=True,
                           max_length=128
                           )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
test_messages[0:6]

In [ ]:
# Datasetleri oluştur
train_dataset = SpamDataset(train_encodings, train_labels)
test_dataset = SpamDataset(test_encodings, test_labels)

In [8]:
# 2 sınıflı (spam/ham) model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##Train

In [9]:
# Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./mBert/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./mBert/logs",
    logging_steps=10
)

# Trainer objesini oluştur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Modeli eğit
trainer.train()
901653a1195d05083d52667fcb966b5e7b4ae1e4

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: salihdegerli2394 (salihdegerli2394-akdeniz-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: salihdegerli2394 (salihdegerli2394-akdeniz-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more informatio

Epoch,Training Loss,Validation Loss
1,0.001300,0.076959
2,0.000200,0.058388
3,0.041800,0.048793


TrainOutput(global_step=3000, training_loss=0.05068106220445285, metrics={'train_runtime': 824.5517, 'train_samples_per_second': 29.107, 'train_steps_per_second': 3.638, 'total_flos': 1578666332160000.0, 'train_loss': 0.05068106220445285, 'epoch': 3.0})

TrainOutput(global_step=3000, training_loss=0.05068106220445285, metrics={'train_runtime': 824.5517, 'train_samples_per_second': 29.107, 'train_steps_per_second': 3.638, 'total_flos': 1578666332160000.0, 'train_loss': 0.05068106220445285, 'epoch': 3.0})

##Validation and Save


In [14]:
# Test setinde tahmin yap
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# Sonuçları değerlendir
accuracy = accuracy_score(test_labels, preds)
f1 = f1_score(test_labels, preds)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1 Score: {f1:.4f}")

# Modeli kaydet
model.save_pretrained("./mBert/spam_detector_model")
tokenizer.save_pretrained("./mBert/spam_detector_model")


Test Accuracy: 0.9925
Test F1 Score: 0.9897


('./mBert/spam_detector_model/tokenizer_config.json',
 './mBert/spam_detector_model/special_tokens_map.json',
 './mBert/spam_detector_model/vocab.txt',
 './mBert/spam_detector_model/added_tokens.json',
 './mBert/spam_detector_model/tokenizer.json')

##Predict

In [15]:
model_path = "./mBert/spam_detector_model"  # Eğittiğin modelin kayıtlı olduğu dizin
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [16]:
# Örnek bir mesaj ile test edelim
message = "i've reached sch already..."
print(f"Message: {message}")
print(f"Predicted: {predict_message(message)}")

Message: i've reached sch already...
Predicted: Ham


In [19]:
root_dir = os.getcwd()
save_dir = os.path.join(root_dir,"mBert/spam_detector_model")
report_zipdir = save_dir+".zip"
!zip -rj $report_zipdir $save_dir
try:
    from google.colab import files
    files.download(report_zipdir)
except ModuleNotFoundError:
    print(f"You can download {report_zipdir}..")

  adding: vocab.txt (deflated 45%)
  adding: tokenizer_config.json (deflated 74%)
  adding: model.safetensors (deflated 7%)
  adding: special_tokens_map.json (deflated 80%)
  adding: config.json (deflated 53%)
  adding: tokenizer.json (deflated 67%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>